# `DEMO` Azure AI Language Service
## Introduction
### Purpose
Use this intsead of storing secrets in config files or in code.

### Authenticate via Visual Studio Code
- [Azure Account extension](https://marketplace.visualstudio.com/items?itemName=ms-vscode.azure-account) (deprecated in January 2025)
- [Install Azure CLI in Windows](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli-windows?tabs=azure-cli)

To run the Azure CLI to authenticate:

```bash
$ az login
```

## Dependencies

In [3]:
# pip install azure-identity
# pip install azure-keyvault-secrets
# !pip install azure-ai-textanalytics

In [4]:
# from azure.keyvault.secrets import SecretClient
# from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

In [5]:
# Language modules
from azure.ai.textanalytics import TextAnalyticsClient

In [37]:
import os
from dotenv import load_dotenv

import pandas as pd

In [7]:
from Azure_Utils import secrets_get

## Retrieve credentials

In [8]:
load_dotenv()

# Retrieve endpoint
endpoint = os.environ["LANG_ENDPOINT"]

# Retrieve key
secretName = os.environ["LANG_SECRET"]
service_key = secrets_get(secretName)

Completed


## Use Azure AI Language service

In [9]:
# Create a client for synchronous operations, using API key authentication
client = TextAnalyticsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(service_key)
)

### Detect language
Documentation: [`detect_language`](https://learn.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-detect-language)

#### Privacy consideration
If `disable_service_logs` is set to `True`, you opt-out of having your text input logged on the service side for troubleshooting. By default, the Language service logs your input text for 48 hours, solely to allow for troubleshooting issues in providing you with the service's natural language processing functions. Setting this parameter to true, disables input logging and may limit our ability to remediate issues that occur.

In [22]:
documents = [
    "Est-ce que tu veux venir au dépanneur pour acheter une poutine?",
    "On se retrouve à la gare pour prendre le train?",
    "Kannst du mir bitte helfen, dieses Problem zu lösen?",
    "Could you grab me a coffee from the store?",
    "Would you fancy a cup of tea this afternoon?",
    "¿Quieres ir a la playa a dar un paseo?",
    "¿Te gustaría ir a comer unos tacos al puesto de la esquina?",
    "هل ترغب في الخروج لتناول القهوة معي؟",
    "Veylun tarë niava lothë senir?",
    "Ĉu vi ŝatus promeni en la parko hodiaŭ?",
    "^%&^*&*"
]

In [23]:
client.detect_language(documents=documents, country_hint='us', show_stats=True, disable_service_logs=True)

[DetectLanguageResult(id=0, primary_language=DetectedLanguage(name=French, iso6391_name=fr, confidence_score=1.0), warnings=[], statistics=TextDocumentStatistics(character_count=63, transaction_count=1), is_error=False),
 DetectLanguageResult(id=1, primary_language=DetectedLanguage(name=French, iso6391_name=fr, confidence_score=1.0), warnings=[], statistics=TextDocumentStatistics(character_count=47, transaction_count=1), is_error=False),
 DetectLanguageResult(id=2, primary_language=DetectedLanguage(name=German, iso6391_name=de, confidence_score=1.0), warnings=[], statistics=TextDocumentStatistics(character_count=52, transaction_count=1), is_error=False),
 DetectLanguageResult(id=3, primary_language=DetectedLanguage(name=English, iso6391_name=en, confidence_score=1.0), warnings=[], statistics=TextDocumentStatistics(character_count=42, transaction_count=1), is_error=False),
 DetectLanguageResult(id=4, primary_language=DetectedLanguage(name=English, iso6391_name=en, confidence_score=1.0),

### Key phrase extraction
Documentation: [`extract_key_phrases`](https://learn.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-extract-key-phrases)

In [11]:
# Use the client key extraction feature
documents = ["Dr. Luke has a very modern medical bureau, and she has great staff."]
response = client.extract_key_phrases(documents = documents)[0]

In [12]:
response['key_phrases']

['modern medical bureau', 'Dr. Luke', 'great staff']

### Analyse sentiment
Documentation: [`analyze_sentiment`](https://learn.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-analyze-sentiment)

In [29]:
client.analyze_sentiment(documents=['hello, you suck,  I like you'], show_opinion_mining=True)

[AnalyzeSentimentResult(id=0, sentiment=positive, warnings=[], statistics=None, confidence_scores=SentimentConfidenceScores(positive=0.55, neutral=0.07, negative=0.38), sentences=[SentenceSentiment(text=hello, you suck,  I like you, sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=0.55, neutral=0.07, negative=0.38), length=28, offset=0, mined_opinions=[])], is_error=False)]

### Extract entities
Documentation: [`recognize_entities`](https://learn.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-recognize-entities)

Many [entities](https://learn.microsoft.com/en-us/azure/ai-services/language-service/named-entity-recognition/concepts/named-entity-categories?tabs=ga-api) are supported.

In [30]:
story = """On January 16, 2025, Emma Johnson attended a conference at the Global Tech Innovation Hub, located at 123 Innovation Drive, San Francisco, CA, 94105. The event, hosted by TechFrontier Inc., featured discussions on advancements in artificial intelligence and natural language processing. Emma, a senior researcher at AI Nexus, was a keynote speaker.
During her presentation, she demonstrated a new tool for recognizing named entities in text, which is crucial for tasks like information extraction and sentiment analysis. After the conference, attendees received an email at info@techfrontier.com containing the presentation slides and a survey link for feedback. Emma also shared additional resources on her personal blog at https://emmajohnsonai.com, where she posts insights into AI research.
Later that evening, Emma had dinner with her colleague Dr. Mark Patel at Golden Fork Bistro in downtown San Francisco. They discussed collaborating on a project focused on applying NER in healthcare data analysis.
If you'd like to know more about TechFrontier’s events, visit their official website at https://www.techfrontier.com, or contact them directly via their office at 555 Technology Avenue, Suite 200, Palo Alto, CA, 94301."""

In [32]:
response = client.recognize_entities(documents=[story])

In [35]:
response[0].keys()

['id', 'entities', 'warnings', 'statistics', 'is_error', 'kind']

In [38]:
pd.DataFrame([{
    'text': e['text'],
    'category': e['category'],
    'subcategory': e['subcategory'],
    'scrore': e['confidence_score']

} for e in response[0]['entities']])

,text,category,subcategory,scrore
0,"January 16, 2025",DateTime,Date,1.00
1,Emma Johnson,Person,None,1.00
2,conference,Event,None,0.91
3,Global Tech Innovation Hub,Organization,None,0.99
4,"123 Innovation Drive, San Francisco, CA, 94105",Address,None,0.96
5,TechFrontier Inc.,Organization,None,0.99
6,artificial intelligence,Skill,None,1.00
7,natural language processing,Skill,None,0.94
8,Emma,Person,None,1.00
9,senior researcher,PersonType,None,0.66


### Extract linked entities
Documentation: [`recognize_linked_entities`](https://learn.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-recognize-linked-entities)

In [39]:
response = client.recognize_linked_entities(documents=[story])

In [40]:
response[0].keys()

['id', 'entities', 'warnings', 'statistics', 'is_error', 'kind']

In [45]:
pd.DataFrame([{
    'name': e['name'],
    'url': e['url'],
    'data_source': e['data_source']
} for e in response[0]['entities']])

,name,url,data_source
0,January 16,https://en.wikipedia.org/wiki/January_16,Wikipedia
1,Emma Johnson (writer),https://en.wikipedia.org/wiki/Emma_Johnson_(wr...,Wikipedia
2,.global,https://en.wikipedia.org/wiki/.global,Wikipedia
3,.tech,https://en.wikipedia.org/wiki/.tech,Wikipedia
4,Innovation Hub,https://en.wikipedia.org/wiki/Innovation_Hub,Wikipedia
5,Innovation,https://en.wikipedia.org/wiki/Innovation,Wikipedia
6,San Francisco,https://en.wikipedia.org/wiki/San_Francisco,Wikipedia
7,PHP,https://en.wikipedia.org/wiki/PHP,Wikipedia
8,Artificial intelligence,https://en.wikipedia.org/wiki/Artificial_intel...,Wikipedia
9,Nexus (data format),https://en.wikipedia.org/wiki/Nexus_(data_format),Wikipedia


In [53]:
# Ambiguous entity
client.recognize_linked_entities(documents=["Venus is beautiful in the sky"])[0]['entities'][0]['url']

'https://en.wikipedia.org/wiki/Venus_(mythology)'

In [52]:
# Ambiguous entity
client.recognize_linked_entities(documents=["Venus is shining in the sky"])[0]['entities'][0]['url']

'https://en.wikipedia.org/wiki/Venus'

### Detect PII
Documentation: [`recognize_pii_entities`](https://learn.microsoft.com/en-us/python/api/azure-ai-textanalytics/azure.ai.textanalytics.textanalyticsclient?view=azure-python#azure-ai-textanalytics-textanalyticsclient-recognize-pii-entities)

Categories ([source](https://learn.microsoft.com/en-us/azure/ai-services/language-service/personally-identifiable-information/concepts/entity-categories)):
- Person
- PersonType
- PhoneNumber
- Organization
- etc.

Filters:
- `domain_filter` = Filters the response entities to ones only included in the specified domain
- `categories_filter`= List of the specific PII entity categories you want to filter out

In [59]:
print(story)

On January 16, 2025, Emma Johnson attended a conference at the Global Tech Innovation Hub, located at 123 Innovation Drive, San Francisco, CA, 94105. The event, hosted by TechFrontier Inc., featured discussions on advancements in artificial intelligence and natural language processing. Emma, a senior researcher at AI Nexus, was a keynote speaker.
During her presentation, she demonstrated a new tool for recognizing named entities in text, which is crucial for tasks like information extraction and sentiment analysis. After the conference, attendees received an email at info@techfrontier.com containing the presentation slides and a survey link for feedback. Emma also shared additional resources on her personal blog at https://emmajohnsonai.com, where she posts insights into AI research.
Later that evening, Emma had dinner with her colleague Dr. Mark Patel at Golden Fork Bistro in downtown San Francisco. They discussed collaborating on a project focused on applying NER in healthcare data a

In [58]:
print(client.recognize_pii_entities(documents=[story])[0]['redacted_text'])

On ****************, ************ attended a conference at the **************************, located at **********************************************. The event, hosted by *****************, featured discussions on advancements in artificial intelligence and natural language processing. ****, a ***************** at ********, was a ***************.
During her presentation, she demonstrated a new tool for recognizing named entities in text, which is crucial for tasks like information extraction and sentiment analysis. After the conference, ********* received an email at ********************* containing the presentation slides and a survey link for feedback. **** also shared additional resources on her personal blog at *************************, where she posts insights into AI research.
Later that evening, **** had dinner with her ********* Dr. ********** at Golden Fork Bistro in downtown San Francisco. They discussed collaborating on a project focused on applying NER in healthcare data a